## Homework 2: Search hyperparameters for neural networks
In this homework, you will practice using Keras to implement neural networks and search their hyperparameters for handwritten digits classification.

### Dataset
MNIST handwritten digits. 60k 28*28 grayscale training images of the 10 digits, along with a test set of 10k images

![Fig. 1](handwrittendigits.png)
 *Fig.1. Handwritten digits examples.* 

### Instructions

    1. You need to install Keras on your computer to build neural networks.
    2. The framework, e.g., functions' names, input, and output, has been defined. You are going to complete the create_NN, nn_params_search, retrain_best_nn, myEvaluation functions.
    3. Add your code in the following blocks, and do not change other places.

```python

    ## add your code here
    
    ##
```

### Student information
    1. Your name: Carlos Santos
    2. Department: Comp. E.
    3. Undergraduate

### Task points and TA grading: ??/100
    1. ?/10
    2. ?/30
    3. ?/30
    4. ?/10
    5. ?/20
    6: performance_acc: ?/10

In [5]:
import keras
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import numpy as np

### 1. Load the MNIST dataset in Keras. 10 points

In [6]:
import matplotlib.pyplot as plt
def load_data():
    '''Load the MNIST dataset'''
    
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
    return (X_train, y_train, X_test, y_test)

#Task 1. load the dataset
(X_train_2D, y_train, X_test_2D, y_test) = load_data()

# 1.1 reshape the training and test sets (N*28*28) to N * 784. 10 points
X_train = np.reshape(X_train_2D, (X_train_2D.shape[0], (X_train_2D.shape[1]*X_train_2D.shape[2])))
X_test = np.reshape(X_test_2D, (X_test_2D.shape[0], (X_test_2D.shape[1]*X_test_2D.shape[2])))

print('X_train_2D: {}, X_train: {}'.format(X_train_2D.shape, X_train.shape))
print('X_test_2D: {}, X_test: {}'.format(X_test_2D.shape, X_test.shape))

# 1.2 transform y_train to one-hot vectors using keras.utils.to_categorical to form the output of the NN. 10 points
y_train_onehot = keras.utils.to_categorical(y_train) 
print('y_train_onehot: {}'.format(y_train_onehot.shape))
print('one-hot vector examples:\n', y_train_onehot[:5])

X_train_2D: (60000, 28, 28), X_train: (60000, 784)
X_test_2D: (10000, 28, 28), X_test: (10000, 784)
y_train_onehot: (60000, 10)
one-hot vector examples:
 [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### 2. Create neural networks using Keras. 30 points

In [7]:
def create_NN(hidden_layers = [1000], act = 'relu', opt = 'rmsprop'): 
    '''create a deep feedforwad neural network using keras
    
    Parameters
    -----------
    hidden_layers: a list that defines the numbers of hidden nodes for all hidden layers, e.g., [1000] indicates
    the nn has only one hidden layer with 1000 nodes, while [1000, 500] defines two hidden layers and the first
    layer has 1000 nodes and the second has 500 nodes.
    act: activation function for all hidden layers
    opt: optimizer
    
    Returns
    -------
    myNN: the neural network model
    
    '''
    in_dim = 784
    out_dim = 10    
    myNN = keras.models.Sequential()
    
    
    num_hidden_layers = 0
    for layer in hidden_layers:
        num_hidden_layers = num_hidden_layers + 1 
        print("Hidden Layer:", num_hidden_layers, "Size of:", layer)
        #2.1 build all hidden layers
        myNN.add(keras.layers.Dense(
        units=layer,
        input_dim=in_dim,
        activation=act))

    #2.2 build the output layer and use the softmax activation  
    myNN.add(keras.layers.Dense(
    units=out_dim,
    activation='softmax'))

    #2.3 choose the optimizer, compile the network and return it. Use 'accuracy' as the metrics
    myNN.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return myNN
    
h_nodes = [1000, 500] # two hidden layers with 1000 and 500 nodes, respectively.
myNN = create_NN(hidden_layers= h_nodes, act = 'relu', opt = 'adam')
myNN.summary()

Hidden Layer: 1 Size of: 1000
Hidden Layer: 2 Size of: 500
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1000)              785000    
_________________________________________________________________
dense_1 (Dense)              (None, 500)               500500    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5010      
Total params: 1,290,510
Trainable params: 1,290,510
Non-trainable params: 0
_________________________________________________________________


### 3. Search the best NN paprameters, and report the performance. 30 points
    - The KerasClassifier will be used to warp NN models to use the GridSearchCV
    - Complete the nn_params_search function to search the three parameters: batch_size, activation, and optimizer.
    - Each fit (10 epochs) may take 1 to 2 minutes.

In [8]:
from keras.wrappers.scikit_learn import KerasClassifier

def nn_params_search(nn, X, y, param_grid): # 30 points
    '''Search best paramaters
    
    Parameters
    ----------
    X_train: features
    y_train: target of the input
    param_grid: a dict that defines the parameters
    
    Returns
    -------
    best_params_
        
    '''
    ## add your code here. set cv = 3, scoring = 'accuracy', and verbose = 2
    grid = GridSearchCV(estimator=nn, cv = 3, scoring = 'accuracy', verbose = 2, param_grid=param_grid)
    results = grid.fit(X, y)
    return results.best_params_
    
    ##

# wrap keras model to use in sklearn
nn = KerasClassifier(build_fn = create_NN, batch_size = 64, epochs = 10) # using the keras wapper
param_grid = {'batch_size': [64, 128], 
              'act':['relu', 'sigmoid'], 'opt': ['sgd', 'adam']}

best_params = nn_params_search(nn, X_train, y_train, param_grid = param_grid)
print('\nBest parameters: ', best_params)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Hidden Layer: 1 Size of: 1000
Epoch 1/10
625/625 [==============================] - 3s 5ms/step - loss: 704.2440 - accuracy: 0.5957
Epoch 2/10
625/625 [==============================] - 5s 7ms/step - loss: 0.6038 - accuracy: 0.8506
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3863 - accuracy: 0.9111
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3169 - accuracy: 0.9274
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2798 - accuracy: 0.9349
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2920 - accuracy: 0.9356
Epoch 7/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2381 - accuracy: 0.9435
Epoch 8/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2414 - accuracy: 0.9440
Epoch 9/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2451 - accuracy: 0.9438
Epoch 10

C:\Users\carlo\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END ...................act=relu, batch_size=64, opt=sgd; total time=  32.8s
Hidden Layer: 1 Size of: 1000
Epoch 1/10
625/625 [==============================] - 3s 5ms/step - loss: 589.7809 - accuracy: 0.6482
Epoch 2/10
625/625 [==============================] - 3s 6ms/step - loss: 0.4913 - accuracy: 0.8928
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3681 - accuracy: 0.9149
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3327 - accuracy: 0.9268
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2358 - accuracy: 0.9424
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2347 - accuracy: 0.9471
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2111 - accuracy: 0.9514
Epoch 8/10
625/625 [==============================] - 3s 6ms/step - loss: 0.1862 - accuracy: 0.9554
Epoch 9/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1663 - accu

C:\Users\carlo\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END ...................act=relu, batch_size=64, opt=sgd; total time=  32.9s
Hidden Layer: 1 Size of: 1000
Epoch 1/10
625/625 [==============================] - 3s 5ms/step - loss: 598.4610 - accuracy: 0.5747
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.7271 - accuracy: 0.8382
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.6391 - accuracy: 0.8533
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.4493 - accuracy: 0.8993
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.4099 - accuracy: 0.9079
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3378 - accuracy: 0.9230
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3070 - accuracy: 0.9318
Epoch 8/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2931 - accuracy: 0.9317
Epoch 9/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2944 - accu

...

[CV] END ..............act=sigmoid, batch_size=128, opt=adam; total time=  28.6s
Hidden Layer: 1 Size of: 1000
Epoch 1/10
313/313 [==============================] - 4s 9ms/step - loss: 0.6021 - accuracy: 0.8277
Epoch 2/10
313/313 [==============================] - 3s 9ms/step - loss: 0.2468 - accuracy: 0.9329
Epoch 3/10
313/313 [==============================] - 3s 8ms/step - loss: 0.2258 - accuracy: 0.9375
Epoch 4/10
313/313 [==============================] - 3s 10ms/step - loss: 0.2012 - accuracy: 0.9437
Epoch 5/10
313/313 [==============================] - 3s 8ms/step - loss: 0.1935 - accuracy: 0.9453
Epoch 6/10
313/313 [==============================] - 3s 9ms/step - loss: 0.1947 - accuracy: 0.9433
Epoch 7/10
313/313 [==============================] - 3s 9ms/step - loss: 0.1887 - accuracy: 0.9477
Epoch 8/10
313/313 [==============================] - 3s 9ms/step - loss: 0.1657 - accuracy: 0.9525
Epoch 9/10
313/313 [==============================] - 3s 8ms/step - loss: 0.1605 - accur

C:\Users\carlo\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END ..............act=sigmoid, batch_size=128, opt=adam; total time=  28.7s
Hidden Layer: 1 Size of: 1000
Epoch 1/10
313/313 [==============================] - 3s 8ms/step - loss: 0.6079 - accuracy: 0.8264
Epoch 2/10
313/313 [==============================] - 3s 8ms/step - loss: 0.2487 - accuracy: 0.9287
Epoch 3/10
313/313 [==============================] - 3s 9ms/step - loss: 0.2135 - accuracy: 0.9404
Epoch 4/10
313/313 [==============================] - 2s 8ms/step - loss: 0.2072 - accuracy: 0.9411
Epoch 5/10
313/313 [==============================] - 2s 7ms/step - loss: 0.1966 - accuracy: 0.9445
Epoch 6/10
313/313 [==============================] - 3s 8ms/step - loss: 0.1822 - accuracy: 0.9476
Epoch 7/10
313/313 [==============================] - 3s 8ms/step - loss: 0.1725 - accuracy: 0.9498
Epoch 8/10
313/313 [==============================] - 2s 8ms/step - loss: 0.1638 - accuracy: 0.9520
Epoch 9/10
313/313 [==============================] - 2s 8ms/step - loss: 0.1637 - accura

C:\Users\carlo\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END ..............act=sigmoid, batch_size=128, opt=adam; total time=  26.2s
Hidden Layer: 1 Size of: 1000
Epoch 1/10
469/469 [==============================] - 4s 7ms/step - loss: 13.1496 - accuracy: 0.8539
Epoch 2/10
469/469 [==============================] - 4s 8ms/step - loss: 0.5478 - accuracy: 0.9599
Epoch 3/10
469/469 [==============================] - 4s 8ms/step - loss: 0.2486 - accuracy: 0.9724
Epoch 4/10
469/469 [==============================] - 4s 9ms/step - loss: 0.1678 - accuracy: 0.9788
Epoch 5/10
469/469 [==============================] - 4s 8ms/step - loss: 0.1760 - accuracy: 0.9772
Epoch 6/10
469/469 [==============================] - 4s 8ms/step - loss: 0.1882 - accuracy: 0.9775
Epoch 7/10
469/469 [==============================] - 4s 8ms/step - loss: 0.1455 - accuracy: 0.9814
Epoch 8/10
469/469 [==============================] - 5s 10ms/step - loss: 0.1936 - accuracy: 0.9780
Epoch 9/10
469/469 [==============================] - 5s 10ms/step - loss: 0.1749 - acc

### 4. Retrain a neural network using the best parameters. 10 points
    - Compelte the retrain_test_nn function to create (create_NN) and train (fit) a new nn using parameters in best_params
    - The default epoches are 20 and each epoch may take around 10 to 20 seconds; and you can increase this value to get better results

In [9]:
def retrain_best_nn(best_params, X_train, y_train, epochs = 10): # 10 points
    '''Retrain a nn using the best parameters
    
    Paramters
    ----------
    best_params:
    X_train: data input of the training set
    y_train: target of the input (one-hot vectors)
    
    Returns
    ---------
    bestNN: the nn classifier trained using the best parameters
    
    '''
    ## add your code here  
    h_nodes = [1000, 500] # two hidden layers with 1000 and 500 nodes, respectively.
    myNN = create_NN(hidden_layers= h_nodes, act = best_params["act"], opt = best_params["opt"])
    myNN.summary()
    results = myNN.fit(X_train, y_train, epochs=epochs, batch_size=best_params["batch_size"])
    return myNN

bestNN = retrain_best_nn(best_params, X_train, y_train_onehot, epochs = 20)

Hidden Layer: 1 Size of: 1000
Hidden Layer: 2 Size of: 500
Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_53 (Dense)             (None, 1000)              785000    
_________________________________________________________________
dense_54 (Dense)             (None, 500)               500500    
_________________________________________________________________
dense_55 (Dense)             (None, 10)                5010      
Total params: 1,290,510
Trainable params: 1,290,510
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
469/469 [==============================] - 9s 17ms/step - loss: 8.8774 - accuracy: 0.8505
Epoch 2/20
469/469 [==============================] - 8s 17ms/step - loss: 0.1828 - accuracy: 0.9601
Epoch 3/20
469/469 [==============================] - 8s 17ms/step - loss: 0.1128 - accuracy: 0.9711
Epoch 4/20


### 5. Network evaluation. 20 points

    - Complete the myEvaluation function to report the performance of your best nn using the test set
        - compute the overall accuracy
        - compute the precision for each class

In [10]:
from sklearn.metrics import classification_report

def myEvaluation(y, y_pred):
    ''' calculate the overall accuracy and precision
    
        Parameters
        ----------
        y: real target
        y_pred: prediction
        
        Returns
        -------
        acc: accuracy
        precision: precision array
    '''

    # calculate the overall acc
    correct_preds = np.sum(y == y_pred, axis=0)
    acc = correct_preds / y.shape[0]
    
    # calculate the precision for each class
    con_mat = confusion_matrix(y, y_pred)
    #print(con_mat)
    con_mat_transpose = con_mat.transpose()
    #print(con_mat_transpose)
    
    
    pre = []
    for num_x, x in enumerate(con_mat_transpose):
        # sum up the column since we transposed it
        col_sum = 0
        for y in x:
            col_sum = col_sum + y  
        col_pre = x[num_x]/col_sum
        pre.append(round(col_pre,2))
    
    # return acc and precision array
    return acc, pre

y_test_pred = bestNN.predict_classes(X_test)
acc, precision = myEvaluation(y_test, y_test_pred)
print('my accuracy:    {:.2f}'.format(acc))
print('my precision:', precision)

# results calculated using the classification_report
print(classification_report(y_test, y_test_pred))

C:\Users\carlo\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


my accuracy:    0.97
my precision: [0.96, 0.99, 0.98, 0.97, 0.97, 0.95, 0.98, 0.97, 0.97, 0.92]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.95      0.96      1032
           3       0.97      0.96      0.97      1010
           4       0.97      0.96      0.97       982
           5       0.95      0.95      0.95       892
           6       0.98      0.96      0.97       958
           7       0.97      0.97      0.97      1028
           8       0.97      0.95      0.96       974
           9       0.92      0.98      0.95      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000



### 6. Any findings or conclusion, e.g., useful strategies to improve nn performance. Extra 10 points
1. To improve performace we can increase the number of epocs, but at the cost of computational time/resources

2. It is possible to provide a larger training set, but again at the cost of computation time/resources. 

3. When we search for the "best parameters", it ensures our "best NN" model will be created with ideal parameters. The "best parameters" will vary depending on the computational resources available under the hood.

